## Data preparation and sanitization

In [1]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import seaborn as sb
import imblearn

from utils.preprocessing import df_to_xydf, df_to_xy

# Read and sanitize the data
df = pd.read_excel("../data/UG_HH_NEW_continuous_no200.xls")
df = df.dropna()

/home/franck/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


ImportError: Install xlrd >= 1.0.0 for Excel support

In [ ]:
df.describe().T

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

# Correlation Matrix Heatmap
f, ax = plt.subplots(figsize=(10, 8))
df_x, df_y = df_to_xydf(df)
df_corr = df_x.copy()
df_corr['min_offer'] = df_y['min_offer']
corr = df_corr.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Features correlation', fontsize=14)

In [ ]:

# Correlation Matrix Heatmap
f, ax = plt.subplots(figsize=(6, 5))
df_x, df_y = df_to_xydf(
    df=df,
    select_columns=['risk', 'selfish', 'time_spent_prop', 'count_effort', 'Honesty_Humility', 'Agreeableness'],
    fuse_risk=True,
)
df_corr = df_x.copy()
df_corr['min_offer'] = df_y['min_offer']
corr = df_corr.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Features* correlation', fontsize=14)

**Pair plots**

In [ ]:
#cols = ['density', 'residual sugar', 'total sulfur dioxide', 'fixed acidity']
pp = sns.pairplot(df_corr, height=1.8, aspect=1.8,
                  plot_kws=dict(edgecolor="k", linewidth=0.5),
                  diag_kind="kde", diag_kws=dict(shade=True))

fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Attributes Pairwise Plots', fontsize=14)

**Target distribution**

In [ ]:
df['min_offer'].hist()

**Proposer offer distribution**

In [ ]:
df['prop'].hist()

In [ ]:
from models.metrics import cross_compute, avg_loss_ratio, gain, gain_mean

print("human avg_loss_ratio", cross_compute(df['min_offer'], df['prop'], avg_loss_ratio))
print("human mean gain", round(cross_compute(df['min_offer'], df['prop'], gain_mean)))

**Responder offer distribution**

In [ ]:
df['other_prop'].hist()

In [ ]:
from models.metrics import cross_compute, avg_loss_ratio, gain, gain_mean

print("human avg_loss_ratio", cross_compute(df['min_offer'], df['other_prop'], avg_loss_ratio))
print("human mean gain", round(cross_compute(df['min_offer'], df['other_prop'], gain_mean)))